In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
from utils.postgres_sql import PostgresSqlConnector

In [3]:
sql_connector = PostgresSqlConnector()

In [4]:
base_ards_ids_path = 'dataset/base_ards_data_and_identification_offset.csv'
base_ards_data = pd.read_csv(base_ards_ids_path, index_col=0)
base_ards_data

,icu_stay_id,identification_offset
0,351515,1771
1,1461683,737
2,3197402,326
3,1652554,586
4,554884,594
...,...,...
668,482078,704
669,1514366,798
670,2296557,541
671,914123,722


In [67]:
query = \
"""
select ards_data.patientunitstayid as icu_stay_id, uniquepid, unitdischargeoffset, unitdischargestatus, hospitaldischargeoffset, hospitaldischargestatus
from ards_data
left join patient p on ards_data.patientunitstayid = p.patientunitstayid
"""
ards_data = sql_connector.get_data_by_query(query)
ards_data = pd.merge(base_ards_data, ards_data, on='icu_stay_id')
ards_data.drop(ards_data[ards_data['unitdischargestatus'] == ''].index, inplace=True)
ards_data.drop(ards_data[(ards_data['unitdischargestatus'] == 'Alive') & (ards_data['hospitaldischargestatus'] == '')].index, inplace=True)
ards_data.sort_values(['uniquepid', 'unitdischargeoffset'], ascending=False)
ards_data.drop_duplicates('uniquepid', keep='first', inplace=True)

ards_data

,icu_stay_id,identification_offset,uniquepid,unitdischargeoffset,unitdischargestatus,hospitaldischargeoffset,hospitaldischargestatus
0,351515,1771,004-15269,45739,Alive,45739,Alive
1,1461683,737,015-14011,4904,Alive,4904,Alive
2,3197402,326,032-5881,2855,Alive,6684,Alive
3,1652554,586,017-61172,2181,Alive,2249,Expired
4,554884,594,006-188102,1638,Expired,1638,Expired
...,...,...,...,...,...,...,...
10463,482078,704,005-79351,2748,Alive,19080,Alive
10464,1514366,798,015-55944,6924,Alive,7061,Expired
10465,2296557,541,022-104717,5344,Alive,9686,Alive
10466,914123,722,006-12067,14250,Alive,14250,Alive


In [70]:
valid_base_data_path = 'dataset/valid_base_ards_data.csv'
ards_data.loc[:, ['icu_stay_id', 'identification_offset', 'uniquepid']].to_csv(valid_base_data_path, index=False)

In [71]:
base_data_path = 'dataset/ards_data_with_static_feature.csv'
ards_static_feature = pd.read_csv(base_data_path, index_col=0)
ards_static_feature

,icu_stay_id,identification_offset,28d_death_status,ards_group,pf_8h_min,admission_diagnosis,gender,age,apacheadmissiondx,admissionheight,...,vasopressor_indicator,dobutamine_indicator,dopamine_indicator,epinephrine_indicator,norepinephrine_indicator,phenylephrine_indicator,vasopressin_indicator,warfarin_indicator,heparin_indicator,milrinone_indicator
0,1716284,556,True,long stay,2.200000,Other,Male,50,Cardiac arrest (with or without respiratory ar...,178.0,...,0,0,0,0,0,0,0,0,0,0
0,3061659,7806,True,long stay,NaN,Other,Female,55,"CHF, congestive heart failure",160.0,...,0,0,0,0,0,0,0,0,0,0
0,1481873,397,False,spontaneous recovery,3.975000,Other,Female,67,Cardiac arrest (with or without respiratory ar...,157.0,...,0,0,0,0,0,0,0,0,0,0
0,919655,551,False,long stay,1.425000,Other,Female,67,"CHF, congestive heart failure",150.0,...,0,0,0,0,0,0,0,0,0,0
0,960449,503,False,long stay,NaN,Pneumonia,Female,65,"Pneumonia, bacterial",162.6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,461302,1252,True,spontaneous recovery,NaN,Cerebrovascular Accident/Stroke,Male,58,"CVA, cerebrovascular accident/stroke",162.6,...,0,0,0,0,0,0,0,0,0,0
0,3217827,623,False,long stay,2.266667,Asthma or Emphysema,Male,54,Emphysema/bronchitis,177.8,...,0,0,0,0,0,0,0,0,0,0
0,739487,4881,False,long stay,1.300000,Respiratory (Medical/Other),Male,75,"Embolus, pulmonary",177.8,...,0,0,0,0,0,0,0,0,0,0
0,2675046,4961,True,long stay,NaN,Asthma or Emphysema,Female,67,Emphysema/bronchitis,167.6,...,0,0,0,0,0,0,0,0,0,0


In [77]:
temp = ards_data.loc[:, ['icu_stay_id', 'uniquepid']]

In [79]:
valid_ards_data_static_feature_path = 'dataset/valid_ards_data_with_static_feature.csv'
pd.merge(temp,ards_static_feature, on='icu_stay_id').to_csv(valid_ards_data_static_feature_path, index=False)